In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
os.chdir('C:/Users/louka/Desktop/Machine Learning course/Project/Machine-learning-project')
print(os.getcwd())

C:\Users\louka\Desktop\Machine Learning course\Project\Machine-learning-project


In [3]:
#useful functions
def swap_rows(df, row1, row2):
    df.iloc[row1], df.iloc[row2] =  df.iloc[row2].copy(), df.iloc[row1].copy()
    return df
def process_percentage_column(column):
    return column.str.rstrip('%').astype(float) / 100

In [128]:
df_train = pd.read_csv('Data/2016 Census AGE, RACE, LITERACY (final version).csv')
df_train_target= pd.read_csv('Data/2016 Presidential Election Results by State.csv')
df_test = pd.read_csv('Data/2020 Census AGE,RACE, LITERACY.csv')
df_test_target=pd.read_csv('Data/2020 elections made.csv')
df_train = df_train.dropna()
df_test=df_test.dropna()
df_train_target=df_train_target.dropna() #weird issue encountered
#print(df_train_target)


#Make indices nice etc
df_train.set_index("STATE",inplace=True) #nice
df_train_target.set_index("STATE",inplace=True)
df_test.set_index("STATE",inplace=True) #nice
df_test_target.set_index("STATE",inplace=True)

#Fix some mistakes
df_test_target = df_test_target.apply(process_percentage_column) #Nice
print(df_test_target)

                      Percent Democrat  Percent Republican  Percent Other
STATE                                                                    
Alabama                         0.3657              0.6203         0.0140
Alaska                          0.4277              0.5283         0.0439
Arizona                         0.4936              0.4906         0.0158
Arkansas                        0.3478              0.6240         0.0283
California                      0.6348              0.3432         0.0220
Colorado                        0.5540              0.4190         0.0270
Connecticut                     0.5924              0.3921         0.0155
Delaware                        0.5878              0.3980         0.0142
District of Columbia            0.9215              0.0540         0.0245
Florida                         0.4786              0.5122         0.0092
Georgia                         0.4950              0.4926         0.0125
Hawaii                          0.6373

In [134]:
df_train_target

,Percentage Democrat,Percentage Republican,Percentage Other
STATE,,,
Alabama,0.34,0.62,0.02
Alaska,0.37,0.51,0.06
Arizona,0.45,0.49,0.04
Arkansas,0.34,0.61,0.03
California,0.62,0.32,0.03
Colorado,0.48,0.43,0.05
Connecticut,0.55,0.41,0.03
Delaware,0.53,0.42,0.03
Dist. of Col.,0.91,0.04,0.02


In [5]:
print(df_test.loc["Alabama"],'\n')
print(df_test_target.loc["Alabama"])

        Median age (years)                                38.358000
Percent under 19                                          25.000000
Percent 19 - 45                                           31.900000
Percent 45-65                                             26.200000
over 65                                                   16.900000
            White                                         67.500000
        Two or more races                                  2.400000
            Black or African American                     26.600000
            American Indian and Alaska Native              0.500000
            Asian                                          1.400000
        Hispanic or Latino (of any race)                   4.400000
            Native Hawaiian and Other Pacific Islander     0.000000
            Some other race                                1.500000
18-24 no highschool                                       12.276557
18-24 high school                               

In [6]:
# df_test.drop(columns=['25+ no highschool'],inplace=True)

In [7]:
# df_train.drop(columns=['25+ no highschool'],inplace=True)

In [27]:
#Perform Kfold in 2016 (train and train_target) data
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
#Set X and y
X = df_train.reset_index(drop=True)
y = df_train_target.reset_index(drop=True)
print(y.shape)
k= 7 #number of folds. 7 is a good candidate for a balanced R^2 mean.
kf = KFold(n_splits=k, shuffle=True, random_state=8) #make kfold item
R2=[]
y_pred_array=[]
for (train_index, test_index) in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    #print(X_train,X_test)
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    print(X_test.index)
    reg=LinearRegression()
    reg.fit(X_train,y_train)
    y_pred=reg.predict(X_test)
    print('R^2= %.2f' %r2_score(y_test,y_pred))
    R2.append(r2_score(y_test,y_pred))
    y_pred_array.append(y_pred)
    print(y_pred,'\n')
    #confusion_matrix(y_test,y_pred)
R2_mean=np.mean(R2)

#print('The mean R² score is: %.2f' %R2_mean)

(51, 4)
Index([0, 1, 22, 25, 31, 35, 49, 50], dtype='int64')


ValueError: could not convert string to float: 'Republican'

In [9]:
reg.coef_

array([[-1.29141477e-02, -3.44823102e-02,  1.51805529e-02,
         1.98868674e-02, -4.68553270e-03, -1.67065163e-01,
        -1.63058910e-01, -1.63643923e-01, -1.74174061e-01,
        -1.82345668e-01,  2.55330674e-03, -7.98633504e-02,
        -1.36178922e-01,  7.97663073e-03, -8.95285129e-03,
        -1.74511766e-03,  6.06862978e+11,  6.06862978e+11,
        -1.50161490e-03],
       [ 9.08224192e-04,  4.92664488e-02, -2.78664247e-03,
        -2.04695630e-03,  3.30566788e-02,  1.64338121e-01,
         1.60964964e-01,  1.61562622e-01,  1.70372900e-01,
         1.83561005e-01, -2.58483343e-03,  6.40937795e-02,
         1.29949140e-01, -6.36755282e-03,  9.37365793e-03,
         6.24560332e-03, -6.19549234e+11, -6.19549234e+11,
        -4.09007161e-04],
       [ 1.00359522e-02,  1.38506493e-03, -6.82506475e-04,
        -2.83599017e-03, -6.07526397e-03,  1.10475714e-02,
         1.15137673e-02,  1.05090987e-02,  1.30927996e-02,
         9.43106165e-03,  2.01045376e-04,  1.60744389e-02,
    

In [10]:
#Now, let's perform a prediction on the whole entire sample
y_tot_pred=reg.predict(df_test)
#print(y_tot_pred)
R2_tot=r2_score(y_tot_pred,df_test_target)
print(R2_tot)

-0.03954772279077501


In [136]:
#XGBoost Regression
X=df_train
y=df_train_target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
estimators = [
    ("imputation_mean", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ('clf', XGBRegressor(random_state=8))
]
pipe=Pipeline(steps=estimators)
pipe

Pipeline(steps=[('imputation_mean', SimpleImputer()),
                ('scaler', StandardScaler()),
                ('clf',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=8, ...))])

In [140]:
search_space = {
    'clf__max_depth': Integer(2,8),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.5, 1.0),
    'clf__colsample_bytree': Real(0.5, 1.0),
    'clf__colsample_bylevel': Real(0.5, 1.0),
    'clf__colsample_bynode' : Real(0.5, 1.0),
    'clf__reg_alpha': Real(0.0, 10.0),
    'clf__reg_lambda': Real(0.0, 10.0),
    'clf__gamma': Real(0.0, 10.0)
}
                                       #cv: number of folds
opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=5, scoring='r2', random_state=8) 
# in reality, you may consider setting cv and n_iter to higher values

In [142]:
opt.fit(X_train, y_train)
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
# Output results

print("Best hyperparameters:", opt.best_params_)
print("R^2 score on the test set:", test_r2)

Best hyperparameters: OrderedDict([('clf__colsample_bylevel', 0.7750018497221565), ('clf__colsample_bynode', 0.5614437441596264), ('clf__colsample_bytree', 0.9126202065825759), ('clf__gamma', 8.289497472648083), ('clf__learning_rate', 0.4299244814327041), ('clf__max_depth', 6), ('clf__reg_alpha', 2.784887532399771), ('clf__reg_lambda', 1.67027558902639), ('clf__subsample', 0.5966102807384807)])
R^2 score on the test set: -7.2189106941223145


In [73]:
#XGBoost Classification
#We need to have y as an array and not a matrix
df_train_target['Winner'] = np.where(
    df_train_target['Percentage Democrat'] < df_train_target['Percentage Republican'],
    'Republican',
    'Democrat'
)
df_test_target['Winner']= np.where(
    df_test_target['Percent Democrat'] < df_test_target['Percent Republican'],
    'Republican',
    'Democrat'
)
# print(df_train_target)
# print(df_train_target['Winner'])


In [33]:

X= df_train
y= df_train_target['Winner'].astype("category")
le = LabelEncoder()
le.fit(y)
y=le.transform(y)
X_train, X_test, y_train, y_test = train_test_split(X.reset_index(drop=True), 
    pd.Series(y).reset_index(drop=True), test_size=0.2,shuffle=True, random_state=8)


In [35]:
print(y)

[1 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1
 0 1 0 1 1 1 1 1 0 0 0 1 0 1]


In [57]:

estimators = [
    ('clf', XGBClassifier(tree_method="hist", enable_categorical=True, device="cuda",random_state=8)) # can customize objective function with the objective parameter
]
pipe = Pipeline(steps=estimators)
pipe

Pipeline(steps=[('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device='cuda',
                               early_stopping_rounds=None,
                               enable_categorical=True, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=8, ...))])

In [67]:

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)
print(confusion_matrix(y_test, y_pred))


y_prob = pipe.predict_proba(X_test)[:, 1]
print(roc_auc_score(y_test, y_prob))

[[20  6]
 [ 2 23]]
0.9500000000000002


In [71]:
X_test= df_test
y_test= df_test_target['Winner'].astype("category")
y_test=le.transform(y_test)
y_pred=pipe.predict(y_test)
y_pred = pipe.predict(X_test)
print(confusion_matrix(y_test, y_pred))


y_prob = pipe.predict_proba(X_test)[:, 1]
print(roc_auc_score(y_test, y_prob))

[[20  6]
 [ 2 23]]
0.9500000000000002


In [39]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
search_space = {
    'clf__max_depth': Integer(2,5),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.5, 1.0),
    'clf__colsample_bytree': Real(0.5, 1.0),
    'clf__colsample_bylevel': Real(0.5, 1.0),
    'clf__colsample_bynode' : Real(0.5, 1.0),
    'clf__reg_alpha': Real(0.0, 10.0),
    'clf__reg_lambda': Real(0.0, 10.0),
    'clf__gamma': Real(0.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=10, scoring='roc_auc', random_state=8) 
# in reality, you may consider setting cv and n_iter to higher values

In [41]:
y_train.shape

(40,)

In [43]:
opt.fit(X, y)

BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('encoder', TargetEncoder()),
                                        ('clf',
                                         XGBClassifier(base_score=None,
                                                       booster=None,
                                                       callbacks=None,
                                                       colsample_bylevel=None,
                                                       colsample_bynode=None,
                                                       colsample_bytree=None,
                                                       device='cuda',
                                                       early_stopping_rounds=None,
                                                       enable_categorical=True,
                                                       eval_metric=None,
                                                       feature_types=None,
                                                       gamma=None,
                                                       grow_policy=None,
                                                       importance_type=...
                             'clf__learning_rate': Real(low=0.001, high=1.0, prior='log-uniform', transform='normalize'),
                             'clf__max_depth': Integer(low=2, high=5, prior='uniform', transform='normalize'),
                             'clf__reg_alpha': Real(low=0.0, high=10.0, prior='uniform', transform='normalize'),
                             'clf__reg_lambda': Real(low=0.0, high=10.0, prior='uniform', transform='normalize'),
                             'clf__subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')})

In [44]:
opt.best_estimator_


Pipeline(steps=[('encoder', TargetEncoder(cols=[])),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=0.5331080642783614,
                               colsample_bynode=0.7262320733948379,
                               colsample_bytree=0.5362052518135637,
                               device='cuda', early_stopping_rounds=None,
                               enable_categorical=True, eval_metric=None,
                               feature_types=None, gamma=7.436578137661654,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.0011002938624638172,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=4, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=8, ...))])

In [45]:
opt.best_score_

0.5

In [46]:
X_test= df_test
y_test= df_test_target['Winner'].astype("category")
y_test=le.transform(y_test)
y_test

array([1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1])

In [47]:
opt.score(X_test, y_test)

0.5

In [48]:
opt.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1])

In [49]:
import xgboost as xgb

#graph = xgb.to_graphviz(opt, num_trees=1)
# Or get a matplotlib axis
#ax = xgb.plot_tree(opt, num_trees=1)
# Get feature importances
opt.best_estimator_.named_steps['clf']

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.5331080642783614,
              colsample_bynode=0.7262320733948379,
              colsample_bytree=0.5362052518135637, device='cuda',
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric=None, feature_types=None, gamma=7.436578137661654,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0011002938624638172,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=8, ...)